In [39]:
!pip3 install newspaper3k

In [40]:
queries, corpus = [], []

In [41]:
!wget https://raw.githubusercontent.com/tassilucas/ir-project/master/data.jsonl

--2022-02-03 06:35:00--  https://raw.githubusercontent.com/tassilucas/ir-project/master/data.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95744 (94K) [text/plain]
Saving to: ‘data.jsonl’

data.jsonl          100%[===================>]  93.50K  --.-KB/s    in 0.01s   

2022-02-03 06:35:00 (6.91 MB/s) - ‘data.jsonl’ saved [95744/95744]



In [42]:
import json

data = []

with open("data.jsonl") as f:
  data = [json.loads(line) for line in f]

In [43]:
y_target = {}

labels = {'Non-relevant': 1, 'Relevant but not useful': 2, 'Relevant and useful': 3, 'Relevant and very useful': 4}

for c in data:
  y_target[c['claim']] = []
  for r in c['results']:
    print("URL: {} / {} -> {}".format(r['url'], r['judgment'], labels[r['judgment']]))
    y_target[c['claim']].append(labels[r['judgment']])
  print('======================================')

URL: https://www.telegraph.co.uk/business/2017/02/28/de-beers-steps-hunt-fake-diamonds-detector-box/ / Relevant but not useful -> 2
URL: http://www.scmp.com/business/companies/article/2076225/de-beers-fights-fakes-technology-chinas-lab-grown-diamonds / Relevant and useful -> 3
URL: https://www.reddit.com/r/Vechain/duplicates/834y0s/china_creates_so_much_synthetic_diamonds_that_are/ / Relevant and useful -> 3
URL: https://news.ycombinator.com/item?id=16551212 / Relevant but not useful -> 2
URL: https://news.ycombinator.com/item?id=16551147 / Non-relevant -> 1
URL: https://www.thediamondloupe.com/technology / Non-relevant -> 1
URL: https://www.thediamondloupe.com/retail-and-consumer-confidence / Non-relevant -> 1
URL: https://www.mining-technology.com/features/featuremining-companies-and-the-war-against-man-made-diamonds-5731454/ / Relevant and useful -> 3
URL: https://insights.abnamro.nl/en/2017/12/diamond-sector-outlook-nothing-is-forever-part-2/ / Relevant and useful -> 3
URL: https:/

In [44]:
from newspaper import Article

texts = {}

for d in data:
  dict_name = d['claim']
  texts[dict_name] = []
  for res in d['results']:
    url = res['url']
    article = Article(url)
    article.download()
    try:
      article.parse()
      print("{} -> {}".format(url, article.meta_description))
    except:
      print("$ Failt to retrieve {}".format(url))

    texts[dict_name].append(article.meta_description + " " + article.text)

  print('--------------------')

https://www.telegraph.co.uk/business/2017/02/28/de-beers-steps-hunt-fake-diamonds-detector-box/ -> Saving up a month’s salary to buy an engagement ring and then discovering the diamond is not real is the stuff of nightmares.
http://www.scmp.com/business/companies/article/2076225/de-beers-fights-fakes-technology-chinas-lab-grown-diamonds -> De Beers fights lab-grown diamonds with technology as China’s synthetic gems threaten viability of their market
https://www.reddit.com/r/Vechain/duplicates/834y0s/china_creates_so_much_synthetic_diamonds_that_are/ -> 25 votes, 35 comments. 221k members in the Vechain community. VeChain is a fully programmable, L1 smart contract platform that connects blockchain …
https://news.ycombinator.com/item?id=16551212 -> 


KeyboardInterrupt: ignored

In [ ]:
texts

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cos_similarity(texts):
  df2 = pd.DataFrame()
  similarities = []

  for query, docs in texts.items():
    df = \
        pd.DataFrame({'text' : docs})

    vectorizer = TfidfVectorizer()

    tfidf = vectorizer.fit_transform(df['text'].values.astype('U'))
    words = vectorizer.get_feature_names()

    queryTFIDF = TfidfVectorizer().fit(words)

    queryTFIDF = queryTFIDF.transform([query])

    similarities.append(cosine_similarity(queryTFIDF, tfidf).flatten())

    similarities_dict = {}

  z = 0
  for query in texts.keys():
    similarities_dict[query] = similarities[z]
    z += 1

  return similarities_dict

In [ ]:
def find_official_sources():
  official_sources = {}

  for d in data:
    name = d['claim']
    official_sources[name] = []
    for res in d['results']:
      if ".gov" in res['url']:
        print("{} is a government/official source.".format(res['url']))
        official_sources[name].append(1)
      else:
        official_sources[name].append(0)
  
  return official_sources

In [ ]:
import re

def find_number_count(texts):
  number_count = {}

  for k, v in texts.items():
    number_count[k] = []
    for doc in v:
      res = re.findall('[0-9]+', doc)
      number_count[k].append(len(res)*0.1)

  # Colocando valores entre [0, 1]
  for k, v in number_count.items():
    number_count[k] = [float(i)/sum(v) for i in v]
  
  return number_count

In [ ]:
def find_freq_relations(texts):
  freq_relations = {}

  for k, v in texts.items():
    freq_relations[k] = []
    for doc in v:
      freq_exclamation = doc.count('!') * -1
      freq_interrogation = doc.count('?') * -1
      freq_relations[k].append(freq_exclamation + freq_interrogation)

  minVal = min(freq_relations[k])
  maxVal = max(freq_relations[k])
  maxRange = 1
  minRange = 0

  # Normalizando valores:
  for k, v in freq_relations.items():
    freq_relations[k] = [(((float(value) - minVal) / float((maxVal- minVal))) *  float((maxRange - minRange)) + minRange) for value in v]
  
  return freq_relations

In [ ]:
!pip install textacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import textacy

def find_quote_count(texts):
  claim_dict = {}

  for k, v in texts.items():

    claim_dict[k] = []
    for t in v:
      doc = textacy.make_spacy_doc(t, lang='en_core_web_sm')
      quote_count_doc = 0
      try:
        res = textacy.extract.triples.direct_quotations(doc)
        for x in list(res):
          quote_count_doc += 1
      except:
        claim_dict[k].append(0)
        continue
      
      claim_dict[k].append(quote_count_doc)
  
  # Colocando valores entre [0, 1]
  for k, v in claim_dict.items():
    claim_dict[k] = [float(i)/sum(v) for i in v]
    
  return claim_dict

In [ ]:
def create_lines(claim, columns):
  new_array = []
  for line_ref in range(0, len(columns[1])):
    line = []
    line.append(claim)
    for col in range(0, len(columns)):
      line.append(columns[col][line_ref])
    line_ref += 1
    new_array.append(line)

  return new_array

last = 0
res = 0
for i, claim_reference_str in enumerate(texts.keys()):
  data_columns = []
  data_columns.append(similarities_dict[claim_reference_str])
  data_columns.append(official_sources[claim_reference_str])
  data_columns.append(number_count[claim_reference_str])
  data_columns.append(freq_relations[claim_reference_str])
  data_columns.append(claim_dict[claim_reference_str])
  data_columns.append(y_target[claim_reference_str])
  data = create_lines(claim_reference_str, data_columns)
  new_frame = pd.DataFrame(data, columns = ['Claim', 'Cosine Similarity (TF-ID)', 'Official Resource', 'Numbers', 'Exclamation/Interrogation', 'Quotes', 'Label'])
  
  if i != 0:
    res = pd.concat([last, new_frame])
    last = res
  else:
    res = new_frame
    last = new_frame

In [ ]:
res

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(res, test_size=0.2, random_state=25)

In [ ]:
train_data

In [ ]:
qid = 1

with open('train.dat', 'w') as td:
  for query in claim_dict.keys():
    d = train_data.loc[train_data['Claim'] == query]
    for index, row in d.iterrows():
      td.write("{} qid:{} 1:{:.1f} 2:{} 3:{:.1f} 4:{:.1f} 5:{:.1f}\n".format(row['Label'], qid, row['Cosine Similarity (TF-ID)'], row['Official Resource'],
                                                                            row['Numbers'], row['Exclamation/Interrogation'], row['Quotes']))
    qid += 1

In [ ]:
!cat train.dat

In [ ]:
test_data

In [ ]:
qid = 1

with open(filename, "w") as test_file:
  for query in claim_dict.keys():
    d = test_data.loc[test_data['Claim'] == query]
    for index, row in d.iterrows():
      test_file.write("{} qid:{} 1:{:.1f} 2:{} 3:{:.1f} 4:{:.1f} 5:{:.1f}\n".format(row['Label'], qid, row['Cosine Similarity (TF-ID)'], row['Official Resource'],
                                                                            row['Numbers'], row['Exclamation/Interrogation'], row['Quotes']))
    qid += 1

In [ ]:
!cat covid-infertility.csv

In [ ]:
import csv
from newspaper import Article

art_dict = {}
test_texts = {}

cos_sim = 0
official_resource = 0
nc = 0
ff = 0
fqc = 0

with open('covid-infertility.csv') as csv_file:
    claim = 'COVID-19 vaccines cause infertility'
    test_texts[claim] = []
    csv_reader = csv.reader(csv_file, delimiter=',')

    labels = []
    for row in csv_reader:
      article = Article(row[0])
      article.download()
      article.parse()
      test_texts[claim].append(article.meta_description + ' ' + article.text)
      labels.append(int(row[1]))

    cos_sim = cos_similarity(test_texts)
    official_resource = 0
    nc = find_number_count(test_texts)
    ff = find_freq_relations(test_texts)
    fqc = find_quote_count(test_texts)

    with open('search_engine.dat', 'w') as se:
      for col in range(0, 16):
        se.write("{} qid:{} 1:{:.1f} 2:{} 3:{:.1f} 4:{:.1f} 5:{:.1f}\n".format(labels[col], 1, cos_sim[claim][col], official_resource, nc[claim][col], ff[claim][col], fqc[claim][col]))

In [ ]:
!cat search_engine.dat

In [ ]:
from google.colab import files
files.download('search_engine.dat')